<a href="https://colab.research.google.com/github/xtenix88/Data-Portfolio/blob/main/Investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import pandas as pd
import numpy as no
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import time
import os

In [56]:
pip install yfinance

In [57]:
import yfinance as yf

In [58]:
# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2023-02-01"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

In [150]:
stock = yf.Ticker("QQQ")
df = stock.history(period="5y")
df.sort_index(axis = 0.,  ascending=False)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-02-10,359.519989,366.488495,356.559998,358.429993,110814955,0.0,0
2022-02-09,363.760010,366.809998,362.119995,366.730011,55766400,0.0,0
2022-02-08,353.959991,360.239990,352.799988,359.119995,64720900,0.0,0
2022-02-07,358.619995,361.049988,353.839996,355.130005,58198100,0.0,0
2022-02-04,354.079987,361.399994,351.970001,358.010010,86127500,0.0,0
...,...,...,...,...,...,...,...
2017-02-17,124.474303,125.207309,124.455010,125.197655,15563700,0.0,0
2017-02-16,124.773263,125.024034,124.339255,124.657532,15098300,0.0,0
2017-02-15,123.876339,124.754009,123.808820,124.696129,17891400,0.0,0


In [71]:
def get_column_from_csv(file, col_name):
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df[col_name]

In [84]:
test = pd.read_csv("/content/stocklist_100_by_volume.csv")
test.head()

,Ticker,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AMC,AMC Entertainment Holdings Inc. Class A Common...,$20.20,1.2608,6.66%,1.038242e+10,NaN,2013.0,61479342.0,Consumer Services,Movies/Entertainment
1,AMD,Advanced Micro Devices Inc. Common Stock,$129.92,-2.9300,-2.21%,1.558140e+11,United States,NaN,55437704.0,Technology,Semiconductors
2,IRNT,IronNet Inc. Common Stock,$4.39,0.8800,25.07%,3.894748e+08,United States,2020.0,54267708.0,Technology,EDP Services
3,SNDL,Sundial Growers Inc. Common Shares,$0.58,0.0129,2.27%,1.198714e+09,Canada,2019.0,52196579.0,NaN,NaN
4,UBER,Uber Technologies Inc. Common Stock,$39.24,-0.9500,-2.36%,7.613024e+10,NaN,2019.0,50929803.0,Technology,EDP Services


In [86]:
tickers = get_column_from_csv("/content/stocklist_100_by_volume.csv", "Ticker")
print(len(tickers))
tickers

8332


0        AMC
1        AMD
2       IRNT
3       SNDL
4       UBER
        ... 
8327     NaN
8328     NaN
8329     NaN
8330     NaN
8331     NaN
Name: Ticker, Length: 8332, dtype: object

In [81]:
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)
    
    try:
        print("Get Data for : ", ticker)
        # Get historical closing price data
        df = stock.history(period="5y")
    
        # Wait 2 seconds
        time.sleep(2)
        
        # Remove the period for saving the file name
        # Save data to a CSV file
        # File to save to 
        the_file = folder + ticker.replace(".", "_") + '.csv'
        print(the_file, " Saved")
        df.to_csv(the_file)
    except Exception as ex:
        print("Couldn't Get Data for :", ticker)

In [82]:
PATH = "/content/"

In [87]:
for x in range(0, 101):
    save_to_csv_from_yahoo(PATH, tickers[x])
    print("Finished")

Get Data for :  AMC
/content/AMC.csv  Saved
Finished
Get Data for :  AMD
/content/AMD.csv  Saved
Finished
Get Data for :  IRNT
/content/IRNT.csv  Saved
Finished
Get Data for :  SNDL
/content/SNDL.csv  Saved
Finished
Get Data for :  UBER
/content/UBER.csv  Saved
Finished
Get Data for :  AREB
/content/AREB.csv  Saved
Finished
Get Data for :  LUMN
/content/LUMN.csv  Saved
Finished
Get Data for :  GRAB
/content/GRAB.csv  Saved
Finished
Get Data for :  AAPL
/content/AAPL.csv  Saved
Finished
Get Data for :  F
/content/F.csv  Saved
Finished
Get Data for :  BBD
/content/BBD.csv  Saved
Finished
Get Data for :  SNAP
/content/SNAP.csv  Saved
Finished
Get Data for :  BKKT
/content/BKKT.csv  Saved
Finished
Get Data for :  TWTR
/content/TWTR.csv  Saved
Finished
Get Data for :  BBIG
/content/BBIG.csv  Saved
Finished
Get Data for :  DIDI
/content/DIDI.csv  Saved
Finished
Get Data for :  NIO
/content/NIO.csv  Saved
Finished
Get Data for :  NVDA
/content/NVDA.csv  Saved
Finished
Get Data for :  SOFI
/co

AttributeError: ignored

In [88]:
def get_stock_df_from_csv(tickers):
  try:
    df = pd.read_csv(PATH + tickers + '.csv', index_col=0)
  except FileNotFoundError:
    print("File Doesn't Exist")
  else:
    return df

In [89]:
get_stock_df_from_csv('AMZN')

,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging
Date,,,,,,,,,,,,,,,,,
2017-02-10,823.820007,828.000000,822.849976,827.460022,2429600,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,843.200012
2017-02-13,831.619995,843.000000,828.549988,836.530029,4172600,0,0,0.010961,1.010961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,848.059998
2017-02-14,837.000000,838.309998,831.450012,836.390015,2792400,0,0,-0.000167,1.010792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,847.380005
2017-02-15,834.000000,842.809998,832.820007,842.700012,2968900,0,0,0.007544,1.018418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,845.609985
2017-02-16,841.840027,845.000000,839.380005,844.140015,2714700,0,0,0.001709,1.020158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,846.820007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-04,3112.129883,3224.000000,3012.159912,3152.790039,12640500,0,0,0.135359,3.810202,3042.370508,3413.000004,2671.741011,2985.185059,3067.520020,3026.352539,3517.734985,NaN
2022-02-07,3170.399902,3253.820068,3145.000000,3158.709961,5131200,0,0,0.001878,3.817357,3037.752002,3399.980197,2675.523807,3000.095093,3067.520020,3033.807556,3517.734985,NaN
2022-02-08,3135.010010,3235.850098,3111.010010,3228.270020,3802000,0,0,0.022022,3.901421,3037.679504,3399.752857,2675.606152,3006.205078,3067.520020,3036.862549,3517.734985,NaN


In [90]:
tickers

0        AMC
1        AMD
2       IRNT
3       SNDL
4       UBER
        ... 
8327     NaN
8328     NaN
8329     NaN
8330     NaN
8331     NaN
Name: Ticker, Length: 8332, dtype: object

In [92]:
from os import listdir
from os.path import isfile, join

files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
len(tickers)

220

In [93]:
def add_daily_return_to_df(df):
  df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
  return df

In [94]:
def add_cum_return_to_df(df):
  df['cum_return'] = (1 + df['daily_return']).cumprod()
  return df

In [95]:
def add_bollinger_bands(df):
  df['middle_band'] = df['Close'].rolling(window=20).mean()
  df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
  df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
  return df

In [96]:
def add_ichimoku(df):
  #Conversion line
  hi_val = df['High'].rolling(window=9).max()
  lo_val = df['Low'].rolling(window=9).min()
  df['Conversion'] = (hi_val+lo_val)/2

  #Base Line
  hi_val2 = df['High'].rolling(window=26).max()
  lo_val2 = df['Low'].rolling(window=26).min()
  df['Baseline'] = (hi_val2+lo_val2)/2

  #Span A
  df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

  #Span B
  hi_val3 = df['High'].rolling(window=52).max()
  lo_val3 = df['Low'].rolling(window=52).min()
  df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)

  #Laggine Span
  df['Lagging'] = df['Close'].shift(-26)
  return df



In [97]:
for x in tickers:
  try:
    print("Working on:", x)
    new_df = get_stock_df_from_csv(x)
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_ichimoku(new_df)
    new_df.to_csv(PATH + x + '.csv')
  except Exception as ex:
    print(ex)

Working on: AFFX
Working on: C
Working on: AFMD
Working on: PCG
Working on: ALIM
Working on: MSFT
Working on: AAPL
Working on: AIXG
Working on: stocklist_100_by_volume
'Close'
Working on: AIRR
Working on: IRNT
Working on: AAXJ
Working on: RIOT
Working on: LU
Working on: XOM
Working on: ADMS
Working on: BEKE
Working on: BABA
Working on: CHPT
Working on: big_stock_sectors
'Close'
Working on: ITUB
Working on: ALDR
Working on: ABCW
Working on: AMZN
Working on: AINV
Working on: FB
Working on: AKAM
Working on: VALE
Working on: ADTN
Working on: ZOM
Working on: COST
Working on: AGNCB
Working on: AGEN
Working on: ACOR
Working on: HOOD
Working on: PTON
Working on: AIQ
Working on: PLUG
Working on: AGII
Working on: SIRI
Working on: AAIT
Working on: ABDC
Working on: AGIIL
Working on: ACSF
Working on: UBER
Working on: ACIW
Working on: AAON
Working on: SWN
Working on: AGND
Working on: ADRE
Working on: ADI
Working on: AEHR
Working on: DCFC
Working on: WFC
Working on: PBR
Working on: ADRA
Working on: A

In [98]:
df = pd.read_csv('/content/AMZN.csv', index_col=0)

#daily return
df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1

#cumulative return
df['cum_return'] = (1 + df['daily_return']).cumprod()

#moving average 20 days
df['middle_band'] = df['Close'].rolling(window=20).mean()
df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()

#Conversion line
hi_val = df['High'].rolling(window=9).max()
lo_val = df['Low'].rolling(window=9).min()
df['Conversion'] = (hi_val+lo_val)/2

#Base Line
hi_val2 = df['High'].rolling(window=26).max()
lo_val2 = df['Low'].rolling(window=26).min()
df['Baseline'] = (hi_val2+lo_val2)/2

#Span A
df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)

#Span B
hi_val3 = df['High'].rolling(window=52).max()
lo_val3 = df['Low'].rolling(window=52).min()
df['SpanB'] = ((hi_val3 + lo_val3)/2).shift(26)

#Laggine Span
df['Lagging'] = df['Close'].shift(-26)

#return df
df


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging
Date,,,,,,,,,,,,,,,,,
2017-02-10,823.820007,828.000000,822.849976,827.460022,2429600,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,843.200012
2017-02-13,831.619995,843.000000,828.549988,836.530029,4172600,0,0,0.010961,1.010961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,848.059998
2017-02-14,837.000000,838.309998,831.450012,836.390015,2792400,0,0,-0.000167,1.010792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,847.380005
2017-02-15,834.000000,842.809998,832.820007,842.700012,2968900,0,0,0.007544,1.018418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,845.609985
2017-02-16,841.840027,845.000000,839.380005,844.140015,2714700,0,0,0.001709,1.020158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,846.820007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-04,3112.129883,3224.000000,3012.159912,3152.790039,12640500,0,0,0.135359,3.810202,3042.370508,3413.000004,2671.741011,2985.185059,3067.520020,3026.352539,3517.734985,NaN
2022-02-07,3170.399902,3253.820068,3145.000000,3158.709961,5131200,0,0,0.001878,3.817357,3037.752002,3399.980197,2675.523807,3000.095093,3067.520020,3033.807556,3517.734985,NaN
2022-02-08,3135.010010,3235.850098,3111.010010,3228.270020,3802000,0,0,0.022022,3.901421,3037.679504,3399.752857,2675.606152,3006.205078,3067.520020,3036.862549,3517.734985,NaN


In [99]:
def plot_with_boll_bands(df, ticker):
  fig = go.Figure()

  candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

  upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
                          line=dict(color='rgba(250,0,0,0.75)',
                          width=1), name='Upper Band')

  mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
                          line=dict(color='rgba(0,0,250,0.75)',
                          width=1), name='Middle Band')
  
  lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
                          line=dict(color='rgba(0,25,0,0.75)',
                          width=1), name='Lower Band')
  
  fig.add_trace(candle)
  fig.add_trace(upper_line)
  fig.add_trace(mid_line)
  fig.add_trace(lower_line)

  fig.update_xaxes(title='Date', rangeslider_visible=True)
  fig.update_yaxes(title='Price')

  fig.update_layout(title="Bollinger Bands", height=1200, width=1800, showlegend=True)

  fig.show()

In [100]:
import plotly.graph_objs as go

fig = go.Figure()

candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

upper_line = go.Scatter(x=df.index, y=df['upper_band'], 
                          line=dict(color='rgba(250,0,0,0.75)',
                          width=1), name='Upper Band')

mid_line = go.Scatter(x=df.index, y=df['middle_band'], 
                          line=dict(color='rgba(0,0,250,0.75)',
                          width=1), name='Middle Band')
  
lower_line = go.Scatter(x=df.index, y=df['lower_band'], 
                          line=dict(color='rgba(0,25,0,0.75)',
                          width=1), name='Lower Band')
  
fig.add_trace(candle)
fig.add_trace(upper_line)
fig.add_trace(mid_line)
fig.add_trace(lower_line)

fig.update_xaxes(title='Date', rangeslider_visible=True)
fig.update_yaxes(title='Price')

fig.update_layout(title="Bollinger Bands", height=1200, width=1800, showlegend=True)

fig.show()

In [101]:
costco = get_stock_df_from_csv('COST')
plot_with_boll_bands(costco, 'COST')

In [102]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [103]:
import numpy as np

In [104]:
def get_ichimoku(df):
  candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')

  df1 = df.copy()
  fig = go.Figure()
  df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0) #return 1 for green, return 0 for red
  df['group'] = df['label'].ne(df['label'].shift()).cumsum()
  df = df.groupby('group')

  dfs = []
  for name, data in df:
    dfs.append(data)
  for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y=df.SpanA,
        line=dict(color='rgba(0,0,0,0)')))

    fig.add_traces(go.Scatter(x=df.index, y=df.SpanB,
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=get_fill_color(df['label'].iloc[0])))
    
  baseline = go.Scatter(x=df1.index, y=df1['Baseline'],
                        line=dict(color='pink', width=2), name='Baseline')
  
  conversion = go.Scatter(x=df1.index, y=df1['Conversion'],
                          line=dict(color='black', width=1), name='Conversion')
  
  lagging = go.Scatter(x=df1.index, y=df1['Lagging'],
                          line=dict(color='purple', width=2), name='Lagging')
  
  span_a = go.Scatter(x=df1.index, y=df1['SpanA'],
                          line=dict(color='green', width=2, dash='dot'), name='Span A')
  
  span_b = go.Scatter(x=df1.index, y=df1['SpanB'],
                          line=dict(color='red', width=1, dash='dot'), name='Span B')
  
  fig.add_trace(candle)
  fig.add_trace(baseline)
  fig.add_trace(conversion)
  fig.add_trace(lagging)
  fig.add_trace(span_a)
  fig.add_trace(span_b)

  fig.update_layout(height=1200, width=1800, showlegend=True)

  fig.show()



In [105]:
costco = get_stock_df_from_csv('COST')
get_ichimoku(costco)

In [106]:
sec_df = pd.read_csv('/content/big_stock_sectors.csv')

In [107]:
sec_df.head()

,Ticker,Name,Description,Mrkt Cap,Sector
0,A,Agilent Technologies,Life Sciences Tools & Services,48.21B,Healthcare
1,AA,Alcoa,Metals & Mining,11.15B,Materials
2,AAC,Ares Acquisition,Blank Check / SPAC,1.22B,SPAC
3,AACG,ATA Creativity Global,Diversified Consumer Services,33.59M,Discretionary
4,AACI,Armada Acquisition I,Blank Check / SPAC,202.75M,SPAC


In [108]:
sec_df['Sector'].value_counts()

Healthcare                1278
Financials                 823
Information Technology     759
Discretionary              599
SPAC                       555
Industrial                 550
Energy                     308
Materials                  285
Real Estate                281
Industrials                237
Communication              179
Staples                     98
Utilities                   91
Services                    54
Name: Sector, dtype: int64

In [109]:
indus_df = sec_df.loc[sec_df['Sector'] == "Industrial"]
health_df = sec_df.loc[sec_df['Sector'] == "Healthcare"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication"]
staple_df = sec_df.loc[sec_df['Sector'] == "Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]


In [110]:
indus_df.head()

,Ticker,Name,Description,Mrkt Cap,Sector
7,AAL,American Airlines Group,Airlines,11.63B,Industrial
11,AAOI,Applied Optoelectronics,Communications Equipment,140.31M,Industrial
12,AAON,"AAON, Inc.",Building Products,4.16B,Industrial
19,AAWW,Atlas Air Worldwide Holdings,Air Freight & Logistics,2.73B,Industrial
21,ABB,ABB Ltd,Electrical Equipment,76.09B,Industrial


In [111]:
sec_df.loc[sec_df['Ticker'] == "DIS"]

,Ticker,Name,Description,Mrkt Cap,Sector
1608,DIS,The Walt Disney Company,Entertainment,281.54B,Communication


In [112]:
sec_df.loc[sec_df['Ticker'] == "COST"]

,Ticker,Name,Description,Mrkt Cap,Sector
1328,COST,Costco Wholesale,Food & Staples Retailing,251.74B,Staples


In [113]:
def get_cum_ret_for_stocks(stock_df):
    tickers = []
    cum_rets = []

    for index, row in stock_df.iterrows():
        df = get_stock_df_from_csv(row['Ticker'])
        if df is None:
            pass
        else:
            tickers.append(row['Ticker'])
            cum = df['cum_return'].iloc[-1]
            cum_rets.append(cum)
    return pd.DataFrame({'Ticker':tickers, 'CUM_RET':cum_rets})

In [114]:

discretion = get_cum_ret_for_stocks(discretion_df)
it = get_cum_ret_for_stocks(it_df)
staple = get_cum_ret_for_stocks(staple_df)
comm = get_cum_ret_for_stocks(comm_df)


File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't Exist
File Doesn't

In [115]:
print(discretion.sort_values(by=['CUM_RET'], ascending=False).head(20))
print(it.sort_values(by=['CUM_RET'], ascending=False).head(20))
print(staple.sort_values(by=['CUM_RET'], ascending=False).head(20))
print(comm.sort_values(by=['CUM_RET'], ascending=False).head(20))

   Ticker    CUM_RET
13   TSLA  17.312894
10    NIO   3.938030
4    DKNG   2.459184
6       F   1.813222
7      KO   1.801736
11   PTON   1.522139
3    CHPT   1.516393
12   RBLX   1.044604
8     MAT   1.001214
1    BBIG   0.621840
0    ABEV   0.602791
9    NCLH   0.494038
2     CCL   0.470599
5     EDU   0.328580
   Ticker   CUM_RET
13    AMD  9.598675
28   NVDA  9.564310
36     SQ  8.127995
37   TWLO  6.870347
33   RIOT  6.829342
0    AAPL  5.609642
26   MSFT  5.150596
17   DDOG  4.802131
2    ACLS  4.648447
10   AGYS  4.572893
7    AEHR  4.550898
3    ADBE  4.333934
27     MU  3.880229
14   AMZN  3.867256
19   DOCU  3.256985
32   PYPL  3.048299
6    ADSK  2.947975
39   TWTR  2.447048
4     ADI  2.412924
5     ADP  2.361558
  Ticker   CUM_RET
1   COST  3.396866
0   ALCO  1.365761
3   PETZ  0.078315
2   FAMI  0.032670
  Ticker   CUM_RET
7   ZNGA  3.602713
2    DIS  1.463878
1  CMCSA  1.452967
4   SIRI  1.414426
0    AMC  0.820524
6      T  0.811416
3   LUMN  0.722421
5   SKLZ  0.469192

In [116]:
df_tech = get_stock_df_from_csv("AMZN")
get_ichimoku(df_tech)

In [117]:
# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2023-02-01"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

risk_free_rate = 0.0125

In [118]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers


['AFFX',
 'C',
 'AFMD',
 'PCG',
 'ALIM',
 'MSFT',
 'AAPL',
 'AIXG',
 'stocklist_100_by_volume',
 'AIRR',
 'IRNT',
 'AAXJ',
 'RIOT',
 'LU',
 'XOM',
 'ADMS',
 'BEKE',
 'BABA',
 'CHPT',
 'big_stock_sectors',
 'ITUB',
 'ALDR',
 'ABCW',
 'AMZN',
 'AINV',
 'FB',
 'AKAM',
 'VALE',
 'ADTN',
 'ZOM',
 'COST',
 'AGNCB',
 'AGEN',
 'ACOR',
 'HOOD',
 'PTON',
 'AIQ',
 'PLUG',
 'AGII',
 'SIRI',
 'AAIT',
 'ABDC',
 'AGIIL',
 'ACSF',
 'UBER',
 'ACIW',
 'AAON',
 'SWN',
 'AGND',
 'ADRE',
 'ADI',
 'AEHR',
 'DCFC',
 'WFC',
 'PBR',
 'ADRA',
 'ALDX',
 'AAL',
 'AFOP',
 'ADES',
 'MAT',
 'SOPA',
 'ACFN',
 'AUY',
 'ALLB',
 'ABIO',
 'KO',
 'RBLX',
 'TWTR',
 'ACFC',
 'TSLA',
 'PYPL',
 'ADVS',
 'XELA',
 'PLTR',
 'LCID',
 'AFSI',
 'ABGB',
 'ACNB',
 'EDU',
 'ALGN',
 'ADSK',
 'AEGN',
 'ABCB',
 'ADMP',
 'NOK',
 'ADHD',
 'ABAC',
 'ACRX',
 'AGYS',
 'ACPW',
 'CLF',
 'PHUN',
 'F',
 'ACHC',
 'AGRX',
 'WISH',
 'ALGT',
 'ALNY',
 'AEIS',
 'ACTA',
 'AEPI',
 'AGNC',
 'LYG',
 'MQ',
 'ABCO',
 'CS',
 'ABMD',
 'SQ',
 'NVDA',
 'ACWX',


In [119]:
def get_stock_df_from_csv(tickers):
  try:
    df = pd.read_csv(PATH + tickers + '.csv', index_col=0)
  except FileNotFoundError:
    print("File Doesn't Exist")
  else:
    return df

In [120]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
  mult_df = pd.DataFrame()

  for x in tickers:
    df = get_stock_df_from_csv(x)
    mask = (df.index >= sdate) & (df.index <=edate)
    mult_df[x] = df.loc[mask][col_name]
  
  return mult_df

In [152]:
port_list = ['COST', 'TSLA', 'BAC', 'DIS', 'PFE', 'AMD', 'UBER', 'FB', 'ADBE']


In [153]:
mult_df = merge_df_by_column_name('Close', S_DATE, E_DATE, *port_list)
mult_df

,COST,TSLA,BAC,DIS,PFE,AMD,UBER,FB,ADBE
Date,,,,,,,,,
2017-02-10,153.726990,53.846001,20.843500,104.629593,25.483242,13.580000,NaN,134.190002,116.849998
2017-02-13,153.691193,56.119999,21.132486,105.003059,25.688049,13.490000,NaN,134.050003,117.650002
2017-02-14,155.183823,56.195999,21.728537,105.654251,25.798338,13.260000,NaN,133.850006,117.580002
2017-02-15,156.408203,55.952000,22.198149,105.510590,26.397011,13.300000,NaN,133.440002,118.730003
2017-02-16,157.078583,53.790001,22.198149,106.018127,26.483665,12.970000,NaN,133.839996,118.930000
...,...,...,...,...,...,...,...,...,...
2022-02-04,519.770020,923.320007,48.279999,142.020004,53.000000,123.599998,37.049999,237.089996,513.539978
2022-02-07,515.890015,907.340027,48.509998,142.509995,53.209999,123.669998,37.540001,224.910004,507.100006
2022-02-08,521.150024,922.000000,49.380001,142.479996,51.700001,128.229996,38.340000,220.179993,511.309998


In [154]:
import plotly.express as px

fig = px.line(mult_df, x=mult_df.index, y=mult_df.columns)
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Price")
fig.update_layout(height=1200, width=1800, 
                  showlegend=True)
fig.show()

In [156]:
mult_cum_df = merge_df_by_column_name('cum_return', S_DATE, E_DATE, *port_list)
mult_cum_df

,COST,TSLA,BAC,DIS,PFE,AMD,UBER,FB,ADBE
Date,,,,,,,,,
2017-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-13,0.999767,1.042232,1.013865,1.003569,1.008037,0.993373,NaN,0.998957,1.006846
2017-02-14,1.009477,1.043643,1.042461,1.009793,1.012365,0.976436,NaN,0.997466,1.006247
2017-02-15,1.017441,1.039112,1.064991,1.008420,1.035858,0.979381,NaN,0.994411,1.016089
2017-02-16,1.021802,0.998960,1.064991,1.013271,1.039258,0.955081,NaN,0.997392,1.017801
...,...,...,...,...,...,...,...,...,...
2022-02-04,3.381124,17.147420,2.316310,1.357360,2.079798,9.101620,0.891268,1.766823,4.394865
2022-02-07,3.355884,16.850648,2.327344,1.362043,2.088039,9.106775,0.903055,1.676056,4.339752
2022-02-08,3.390101,17.122906,2.369084,1.361756,2.028784,9.442562,0.922300,1.640808,4.375781


In [157]:
fig = px.line(mult_cum_df, x=mult_cum_df.index, y=mult_cum_df.columns)
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Price")
fig.update_layout(height=1200, width=1800, 
                  showlegend=True)
fig.show()